In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Load data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# Preview the first few rows
train.head()

